## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key


#Google API Key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/lukeperrin/Documents/UCB Data Bootcamp/github-repositories/06-World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,67.28,71,100,8.88,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,79.12,74,3,13.13,light rain
2,2,Valparaiso,CL,-33.0393,-71.6273,58.64,72,0,4.61,clear sky
3,3,Devils Lake,US,48.1128,-98.8651,28.94,76,100,16.11,overcast clouds
4,4,Ponta Do Sol,PT,32.6667,-17.1000,62.01,77,88,6.40,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 65


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
raw_preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
raw_preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Valparaiso,CL,-33.0393,-71.6273,58.64,72,0,4.61,clear sky
4,4,Ponta Do Sol,PT,32.6667,-17.1000,62.01,77,88,6.40,overcast clouds
12,12,Westfield,US,42.1251,-72.7495,61.18,86,100,19.57,overcast clouds
17,17,Punta Arenas,CL,-53.1500,-70.9167,50.11,87,0,20.71,clear sky
21,21,New Norfolk,AU,-42.7826,147.0587,63.66,36,77,3.00,broken clouds
22,22,Praia Da Vitoria,PT,38.7333,-27.0667,57.76,76,40,12.75,scattered clouds
61,61,Camacha,PT,33.0833,-16.3333,58.53,72,40,13.80,scattered clouds
64,64,Lebu,CL,-37.6167,-73.6500,55.35,88,25,2.95,scattered clouds
65,65,Taoudenni,ML,22.6783,-3.9836,64.18,22,0,11.12,clear sky
81,81,Ayna,PE,-12.6500,-73.9167,61.84,96,99,0.63,light rain


In [5]:
# 4a. Determine if there are any empty rows.
raw_preferred_cities_df.count()

City_ID                95
City                   95
Country                94
Lat                    95
Lng                    95
Max Temp               95
Humidity               95
Cloudiness             95
Wind Speed             95
Current Description    95
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = raw_preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
# hotel_df.head(10)

In [8]:
# # 6a. Set parameters to search for hotels with 5000 meters.
# params = {
#     "radius": 5000,
#     "type": "lodging",
#     "key": g_key
# }
#
# # 6b. Iterate through the hotel DataFrame.
# # for i, j in hotel_df.iterrows():
#     # 6c. Get latitude and longitude from DataFrame
# lat = hotel_df['Lat']
# lng = hotel_df['Lng']
# location = f"{lat},{lng}"
#
# # 6d. Set up the base URL for the Google Directions API to get JSON data.
# gmap_nearbysearch_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
# hotel_response = requests.get(gmap_nearbysearch_url, params=params).json()
# hotel_response

In [9]:
# 6b. Iterate through the hotel DataFrame.
for i, j in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = j['Lat']
    lng = j['Lng']
    location = f"{lat},{lng}"
    # 6a. Set parameters to search for hotels with 5000 meters.
    params = {
        "radius": 5000,
        "type": "lodging",
        "key": g_key,
        "location": location
    }
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    gmap_nearbysearch_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_response = requests.get(gmap_nearbysearch_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotel_response["results"][0]["name"]
        print("Found Hotel: " + hotel_response["results"][0]["name"])
    except (IndexError):
        print("No hotel found...skipping.")
        

Found Hotel: Hotel Ultramar
Found Hotel: Hotel do Campo
Found Hotel: Quality Inn
Found Hotel: Hotel Hain
Found Hotel: The Shingles Riverside Cottages
Found Hotel: Hotel Branco I
Found Hotel: Hotel Porto Santo & Spa
Found Hotel: Hostal Las Lilas
No hotel found...skipping.
No hotel found...skipping.
Found Hotel: City Garden Renqiu Meishicheng Store
Found Hotel: departamento comodo
Found Hotel: Quinta da Meia Eira
Found Hotel: Gran Hotel Mar del Plata
Found Hotel: Hotel San Juan
Found Hotel: Hotel Winburg
Found Hotel: St Ives Hobart Accommodation Tasmania
Found Hotel: Blue Bay Lodge
Found Hotel: HOSTAL MARIAN
Found Hotel: Super 8 by Wyndham Fortuna
Found Hotel: Pestana Bahia Praia
Found Hotel: Don Eddie's Sport Fishing Center
Found Hotel: Hôtel Abda
Found Hotel: Courtyard by Marriott Danbury
Found Hotel: Nuevo Hotel Plaza
Found Hotel: صاحبة الفخامة
Found Hotel: Hotel El Araucano
Found Hotel: Jewel Inn Beni Suef
Found Hotel: AQUA Hotel Aquamarina & Spa
Found Hotel: Holiday Inn Express & Su

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{City}</dd>
<dt>City</dt><dd>{Country}</dd>
<dt>Country</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))